In [1]:
git_repo = "https://huggingface.co/datasets/ShengbinYue/DISC-Law-SFT.git"

In [2]:
!git clone {git_repo}

fatal: destination path 'DISC-Law-SFT' already exists and is not an empty directory.


In [3]:
%%bash

pushd "DISC-Law-SFT"
git lfs install
git lfs pull
popd

/data/huengchi/Research/OpenFedLLM/datasets/DISC-Law-SFT/DISC-Law-SFT /data/huengchi/Research/OpenFedLLM/datasets/DISC-Law-SFT
Updated git hooks.
Git LFS initialized.
/data/huengchi/Research/OpenFedLLM/datasets/DISC-Law-SFT


In [4]:
import pandas as pd
import os

In [5]:
ds = pd.read_json('DISC-Law-SFT/DISC-Law-SFT-Pair.jsonl', lines=True)

In [6]:
ds

,id,input,output
0,jud_doc_sum-1,请大致描述这篇文书的内容。\n\n唐鲜明与何伟华、深圳市华名威电汽车服务有限公司侵权责任纠纷...,总结：原告与被告侵权责任纠纷一案。原告提出诉求：判决被告赔偿原告医疗费、住院伙食补助费、营养...
1,jud_doc_sum-2,这是一篇法律文书\n阳晓红、宁运豪等与许绪发、许先平等租赁合同纠纷一审民事判决书\n湖南省衡...,总结：本案系租赁合同纠纷。原告提出诉求，请求判令三被告支付租金、违约金及逾期利息损失。被告许...
2,jud_doc_sum-3,苏州市春秋汽车服务有限公司、周志斗与岳治国租赁合同纠纷一审民事判决书\n江苏省阜宁县人民法院...,总结：本案系租赁合同纠纷。原告提出诉求要求被告偿还租车费及逾期利息。被告未作答辩。经审理原被...
3,jud_doc_sum-4,请归纳这篇文书的大致要点：\n\n闫志钢与天津晶悦酒店有限公司侵权责任纠纷一审民事判决书\n...,总结：原告与被告侵权责任纠纷一案。原告提出诉求：1、被告支付原告工资损失2、被告赔偿原告养老...
4,jud_doc_sum-5,广州市某数码科技有限公司与曾某劳动合同纠纷一审民事判决书\n广东省广州市番禺区人民法院\n民...,原被告系劳动合同纠纷。原告诉称应支付被告代通知金4102.68元、经济补偿金9680.84元...
...,...,...,...
166753,sim_case_match-8134,"根据案例A的内容,判断A更接近于案例B还是案例C,结果以{'LABEL': B_C}的格式给...","{""LABEL"":""C""}"
166754,sim_case_match-8135,"根据案例A的内容,判断A更接近于案例B还是案例C,结果以{'LABEL': B_C}的格式给...","{""LABEL"":""B""}"
166755,sim_case_match-8136,"根据案例A的内容,判断A更接近于案例B还是案例C,结果以{'LABEL': B_C}的格式给...","{""LABEL"":""B""}"
166756,sim_case_match-8137,"根据案例A的内容,判断A更接近于案例B还是案例C,结果以{'LABEL': B_C}的格式给...","{""LABEL"":""B""}"


In [7]:
task_ds = {}
for task_name in set(ds['id'].apply(lambda s: s.rsplit('-')[0])):
    task_ds[task_name] = ds[ds['id'].apply(lambda s: s.startswith(task_name))]
task_ds.keys()

dict_keys(['leg_ele_extra', 'jud_read_compre', 'sent_pred', 'op_sum', 'jud_doc_sum', 'exam', 'sim_case_match', 'leg_case_cls', 'leg_eve_detec'])

In [8]:
for task_name, task_df in task_ds.items():
    saving_dir = os.path.join(task_name, f'{task_name}.jsonl')
    os.makedirs(task_name, exist_ok=True)
    task_df.to_json(saving_dir, orient='records', lines=True)
    print(f'{saving_dir} saved.')

leg_ele_extra/leg_ele_extra.jsonl saved.
jud_read_compre/jud_read_compre.jsonl saved.
sent_pred/sent_pred.jsonl saved.
op_sum/op_sum.jsonl saved.
jud_doc_sum/jud_doc_sum.jsonl saved.
exam/exam.jsonl saved.
sim_case_match/sim_case_match.jsonl saved.
leg_case_cls/leg_case_cls.jsonl saved.
leg_eve_detec/leg_eve_detec.jsonl saved.


In [9]:
test_set_ratio = 0.1

In [10]:
for task_name, task_df in task_ds.items():
    train_set_saving_dir = os.path.join(task_name, f'{task_name}_train_split.jsonl')
    test_set_saving_dir = os.path.join(task_name, f'{task_name}_test_split.jsonl')
    
    test_size = int(task_df.shape[0]*test_set_ratio)

    task_df[:-test_size].to_json(train_set_saving_dir, orient='records', lines=True)
    task_df[-test_size:].to_json(test_set_saving_dir, orient='records', lines=True)

    print(f'{train_set_saving_dir} saved.')
    print(f'{test_set_saving_dir} saved.')

leg_ele_extra/leg_ele_extra_train_split.jsonl saved.
leg_ele_extra/leg_ele_extra_test_split.jsonl saved.
jud_read_compre/jud_read_compre_train_split.jsonl saved.
jud_read_compre/jud_read_compre_test_split.jsonl saved.
sent_pred/sent_pred_train_split.jsonl saved.
sent_pred/sent_pred_test_split.jsonl saved.
op_sum/op_sum_train_split.jsonl saved.
op_sum/op_sum_test_split.jsonl saved.
jud_doc_sum/jud_doc_sum_train_split.jsonl saved.
jud_doc_sum/jud_doc_sum_test_split.jsonl saved.
exam/exam_train_split.jsonl saved.
exam/exam_test_split.jsonl saved.
sim_case_match/sim_case_match_train_split.jsonl saved.
sim_case_match/sim_case_match_test_split.jsonl saved.
leg_case_cls/leg_case_cls_train_split.jsonl saved.
leg_case_cls/leg_case_cls_test_split.jsonl saved.
leg_eve_detec/leg_eve_detec_train_split.jsonl saved.
leg_eve_detec/leg_eve_detec_test_split.jsonl saved.
